# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env 
# (e.g., MinIO storage path) resolve correctly
%cd ../

In [ ]:
import sys
sys.path.append('.')

from pydantic import ValidationError
from botocore.exceptions import ClientError
import aiohttp
from app.async_client import documents_storage_client, images_storage_client
from app.schemas import (
    UploadFileRequest, 
    DownloadFileRequest, 
    DeleteFileRequest, 
    GetFileMetadataRequest, 
    PresignedPutURLRequest, 
    PresignedGetURLRequest
)

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'Kant_the-critique-of-pure-reason.pdf' instead of 'assets/Kant_the-critique-of-pure-reason.pdf')
%cd playground-testing/assets/

# **Test**

## **I. Test `documents_storage_client`**

### **i. Test `upload()` method**

In [ ]:
async def test_upload_file(storage_key: str, file_path: str):
    try:
        request = UploadFileRequest(storage_key=storage_key, file_path=file_path)
        await documents_storage_client.upload(request)
        print(f"File '{storage_key}' uploaded successfully to bucket '{documents_storage_client.bucket_name}'.")
        return True
        
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during upload ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during upload: {e}")
        return False

In [ ]:
await test_upload_file(storage_key="kant-book", file_path="Kant_the-critique-of-pure-reason.pdf")

In [ ]:
# Attempting to upload a non-existent file
await test_upload_file(storage_key="tarski-logic-book", file_path="Tarski1948")

In [ ]:
# Attempting to upload file with wrong mime-type
await test_upload_file(storage_key="tarski-logic-book", file_path="test-image.jpg")

### **ii. Test `download()` method**

In [ ]:
async def test_download_file(storage_key: str, file_path: str):
    try:
        request = DownloadFileRequest(storage_key=storage_key, file_path=file_path)
        await documents_storage_client.download(request)
        print(f"File '{storage_key}' downloaded successfully to '{file_path}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during download ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during download: {e}")
        return False

In [ ]:
await test_download_file(storage_key = "kant-book", file_path = "export.pdf")

In [ ]:
# Attempting to download a non-existent file
await test_download_file(storage_key = "hello-there", file_path = "export.pdf")

### **iii. Test `get_metadata()` method**

In [ ]:
async def test_get_file_metadata(storage_key: str):
    try:
        request = GetFileMetadataRequest(storage_key=storage_key)
        metadata = await documents_storage_client.get_metadata(request)
        print(f"Metadata retrieved for '{storage_key}':")
        print(metadata)
        return True

    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during retrieval ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during retrieval: {e}")
        return False

In [ ]:
await test_get_file_metadata(storage_key = "kant-book")

In [ ]:
# Attempting to get metadata for a non-existent file
await test_get_file_metadata(storage_key = "do-not-exist")

### **iv. Test `delete()` method**

In [ ]:
async def test_delete_file(storage_key: str):
    try:
        request = DeleteFileRequest(storage_key=storage_key)
        await documents_storage_client.delete(request)
        print(f"File '{storage_key}' deleted (or did not exist) from bucket '{documents_storage_client.bucket_name}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False

    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during delete ({error_code}): {e}")
        return False
    
    except Exception as e:
        print(f"Unexpected error during delete: {e}")
        return False

In [ ]:
await test_delete_file(storage_key = "kant-book")

In [ ]:
# Attempting to delete a non-existent file
await test_delete_file(storage_key = "hello-there!")

### **v. Test `generate_presigned_put_url()` method**

In [ ]:
async def test_generate_presigned_put_url(storage_key: str, content_type: str, expires: int = 3600):
    try:
        request = PresignedPutURLRequest(
            storage_key=storage_key,
            content_type=content_type,
            expires=expires
        )
        url = await documents_storage_client.generate_presigned_put_url(request)
        print(f"Presigned PUT URL generated for '{storage_key}' (expires in {expires}s).")
        return url

    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False

    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during presigned URL generation ({error_code}): {e}")
        return False

    except Exception as e:
        print(f"Unexpected error during presigned PUT URL generation: {e}")
        return False

In [ ]:
url = await test_generate_presigned_put_url(storage_key = "new-kant", content_type = "application/pdf")

In [ ]:
# Attempting to generate presigned put url for file with wrong mime-type
await test_generate_presigned_put_url(storage_key = "new-kant", content_type = "image/jpeg")

### **vi. Test `generated-presigned-put-url`**

In [ ]:
async def test_generated_presigned_put_url(
    presigned_url: str,
    file_path: str,
    content_type: str
) -> bool:

    try:
        async with aiohttp.ClientSession() as session:
            with open(file_path, 'rb') as f:
                headers = {'Content-Type': content_type}
                async with session.put(presigned_url, data=f, headers=headers) as response:
                    if response.status == 200:
                        print(f"File '{file_path}' successfully uploaded via presigned URL.")
                        return True
                    else:
                        print(f"Upload failed. HTTP {response.status}: {await response.text()}")
                        return False

    except Exception as e:
        print(f"Unexpected error during upload: {e}")
        return False

In [ ]:
await test_generated_presigned_put_url(presigned_url = url, file_path = "Kant_the-critique-of-pure-reason.pdf", content_type = "application/pdf")

In [ ]:
# Attempting to upload with a mismatched Content-Type
await test_generated_presigned_put_url(presigned_url = url, file_path = "Kant_the-critique-of-pure-reason.pdf", content_type = "image/jpeg")

### **vii. Test `generate_presigned_get_url()` method**

In [ ]:
async def test_generate_presigned_get_url(storage_key: str, expires: int = 3600):
    try:
        request = PresignedGetURLRequest(
            storage_key=storage_key,
            expires=expires
        )
        url = await documents_storage_client.generate_presigned_get_url(request)
        print(f"Presigned GET URL generated for '{storage_key}' (expires in {expires}s).")
        return url

    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False

    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during presigned URL generation ({error_code}): {e}")
        return False

    except Exception as e:
        print(f"Unexpected error during presigned GET URL generation: {e}")
        return False

In [ ]:
url = await test_generate_presigned_get_url(storage_key = "new-kant")

In [ ]:
# Attempting to generate presigned get url for a non-existent file
await test_generate_presigned_get_url(storage_key = "wrong-name")

### **viii. Test `generated-presigned-get-url`**

In [ ]:
async def test_generated_presigned_get_url(
    presigned_url: str,
    save_path: str
) -> bool:
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(presigned_url, timeout=aiohttp.ClientTimeout(total=30)) as response:
                if response.status == 200:
                    with open(save_path, 'wb') as f:
                        async for chunk in response.content.iter_chunked(8192):
                            if chunk:
                                f.write(chunk)
                    print(f"File successfully downloaded and saved to '{save_path}'.")
                    return True
                else:
                    print(f"Download failed. HTTP {response.status}: {await response.text()}")
                    return False

    except Exception as e:
        print(f"Unexpected error during download: {e}")
        return False

In [ ]:
await test_generated_presigned_get_url(presigned_url = url, save_path = "export.pdf")

In [ ]:
# Attempting to download file from wrong get url
await test_generated_presigned_get_url(presigned_url = "https://non-existing-url", save_path = "export.pdf")

## **II. Test `images_storage_client`**

### **Note**

Both `documents_storage_client` and `images_storage_client` are instances of the same `ObjectStorageClient` class and share identical logic. They differ only in configuration: bucket name, max file size, allowed mime types, and expiration days. 

The expiration days value is actively applied at startup via `setup_lifecycle()`, which configures a real bucket lifecycle rule in MinIO/S3. If this failed, the service would crash on launch — so successful startup confirms it works. 

Likewise, while max file size and allowed mime types have different values per client, the validation logic is identical and shared. 

Therefore, testing all core methods on `documents_storage_client` fully covers the common implementation. The only exception is file size testing: large images are easier to source than large documents, so we use `images_storage_client` specifically for max file size boundary checks.

In [ ]:
async def test_upload_large_file(storage_key: str, file_path: str):
    try:
        request = UploadFileRequest(storage_key=storage_key, file_path=file_path)
        await images_storage_client.upload(request)
        print(f"File '{storage_key}' uploaded successfully to bucket '{images_storage_client.bucket_name}'.")
        return True
        
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during upload ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during upload: {e}")
        return False

In [ ]:
await test_upload_large_file(storage_key="image", file_path="test-image.jpg")

In [ ]:
await test_upload_large_file(storage_key="image", file_path="large-image.png")